In [32]:
# basic chatbot using langgraph
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama 

In [33]:
class State(TypedDict):
    # messages have the type "list", 
    # the add_messages function Annotated list defines how this state key should be updated , in this case, it should append a message to the list rather than overwriting them
    messages: Annotated[list, add_messages]

In [34]:
# create a StateGraph instance and pass it our State Class. We will use this to create nodes/edges later on.
graph_builder = StateGraph(State)

In [35]:
llm = ChatOllama(model="llama3.2:3b")

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# now add a node, first argument is the unique node name, second argumnet is the function or object that will be called whenever the node is used

graph_builder.add_node("chatnode", chatbot)
graph_builder.add_edge(START, "chatnode")
graph_builder.add_edge("chatnode", END)

# finally we compile our graph before we can use it
graph = graph_builder.compile()

In [36]:
######################RANDOM DATE##########################
import random
import time
    
def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%m/%d/%Y %I:%M %p', prop)

In [ ]:
def stream_graph_updates(user_input:str):
    for event in graph.stream({"messages": [{"role":"user", "content":user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "cheers", "q"]:
            break

        stream_graph_updates(user_input)
    except:
        # a fallback prompt if the user enters nothing
        date = random_date("1/1/1995 1:30 AM", "1/1/2025 4:50 AM", random.random())
        print(f"what was random date? {date}")
        default_input = f"What were you doing on the night of the {date}?" 
        stream_graph_updates(default_input)
        break